In [1]:
!pip install uritools

In [2]:
import pandas as pd
import numpy as np
import boto3
import pyarrow.parquet as pq
import s3fs
import math
import yaml

import sys
sys.path.append('../../')
from src.utils import read_multipart_parquet_s3, write_df_to_parquet_on_s3

In [14]:
version_ = {
    '20211111_001':['predict-2021-11-11-02-11-40-991.parquet.out'],
    '20211121_002':['predict-2021-11-21-13-55-13-776.parquet.out'],
    '20211121_101':['predict-2021-11-21-14-39-27-332.parquet.out'],
    '20211123_102':['predict-2021-11-23-01-35-04-699.parquet.out','predict-2021-11-25-08-22-59-160.parquet.out'],
    '20211123_103':['predict-2021-11-23-07-55-26-180.parquet.out','predict-2021-11-25-07-41-23-812.parquet.out'],
    '20211123_104':['predict-2021-11-23-14-02-04-205.parquet.out'],
    '20211126_105':['predict-2021-11-26-00-31-33-209.parquet.out'],
    '20211129_106':['predict-2021-11-29-15-02-24-595.parquet.out','predict-2021-11-29-15-48-07-635.parquet.out'],
    '20211201_107':['predict-2021-12-01-12-42-18-742.parquet.out','predict-2021-12-01-13-48-08-809.parquet.out'],
    '20211208_108':['predict-2021-12-08-09-20-13-444.parquet.out','predict-2021-12-08-11-46-24-787.parquet.out','predict-2021-12-09-01-56-35-039.parquet.out'],
    '20211213_109':['predict-2021-12-13-04-22-12-835.parquet.out'],
    '20211214_110':['predict-2021-12-14-12-15-12-419.parquet.out','predict-2021-12-16-06-54-32-471.parquet.out'],
    '20211214_111':['predict-2021-12-14-08-38-28-448.parquet.out','predict-2021-12-17-08-05-49-494.parquet.out'],
    '20211214_112':['predict-2021-12-14-12-15-12-419.parquet.out','predict-2021-12-14-08-38-28-448.parquet.out',
                    'predict-2021-12-16-06-54-32-471.parquet.out','predict-2021-12-17-08-05-49-494.parquet.out'],
    '20211215_113':['predict-2021-12-15-08-42-35-982.parquet.out'],
    '20211215_114':['predict-2021-12-15-10-36-39-946.parquet.out'],
    '20211215_115':['predict-2021-12-15-10-36-39-946.parquet.out','predict-2021-12-15-08-42-35-982.parquet.out'],
    '20211222_116':['predict-2021-12-22-09-31-03-818.parquet.out'],
    '20211222_117':['predict-2021-12-22-12-14-27-758.parquet.out'],
    '20211222_118':['predict-2021-12-22-09-31-03-818.parquet.out','predict-2021-12-22-12-14-27-758.parquet.out'],
    '20211223_119':['predict-2021-12-22-23-52-45-032.parquet.out'],
    '20211223_120':['predict-2021-12-23-01-47-00-753.parquet.out'],
    '20211223_121':['predict-2021-12-22-23-52-45-032.parquet.out','predict-2021-12-23-01-47-00-753.parquet.out'],
    '20211223_122':['predict-2021-12-23-08-35-10-881.parquet.out'],
    '20211224_123':['predict-2021-12-24-01-15-25-075.parquet.out'],
    '20211227_124':['predict-2021-12-27-04-31-39-626.parquet.out'],
    '20211227_125':['predict-2021-12-27-07-57-03-675.parquet.out'],
    '20211228_126':['predict-2021-12-28-03-36-03-673.parquet.out'],
    '20211228_127':['predict-2021-12-28-05-59-55-380.parquet.out'],
    '20211229_128':['predict-2021-12-29-07-19-03-898.parquet.out'],
    '20211230_129':['predict-2021-12-30-01-59-41-184.parquet.out'],
    '20211230_130':['predict-2021-12-30-08-01-55-635.parquet.out'],
    '20211230_131':['predict-2021-12-30-13-56-27-316.parquet.out'],
    '20220104_132':['predict-2022-01-04-00-20-50-192.parquet.out'],
    '20220105_133':['predict-2022-01-05-03-10-10-740.parquet.out'],
    '20220105_134':['predict-2022-01-05-04-53-52-866.parquet.out'],
    '20220105_135':['predict-2022-01-05-03-10-10-740.parquet.out','predict-2022-01-05-04-53-52-866.parquet.out'],
    '20220105_136':['predict-2022-01-05-11-21-19-925.parquet.out'],
    '20220105_137':['predict-2022-01-05-08-38-25-968.parquet.out'],
    '20220105_138':['predict-2022-01-05-11-21-19-925.parquet.out','predict-2022-01-05-08-38-25-968.parquet.out'],
    '20220106_139':['predict-2022-01-06-13-53-54-169.json.out'],
    '20220109_140':['predict-2022-01-09-09-52-37-843.json.out'],
    '20220109_141':['predict-2022-01-10-06-18-48-430.parquet.out'],
    '20220110_142':['predict-2022-01-10-13-24-25-076.parquet.out'],
    '20220113_143':['predict-2022-01-13-04-25-04-055.parquet.out'],
    '20220113_144':['predict-2022-01-13-10-44-07-342.parquet.out'],
    '20220113_145':['predict-2022-01-13-04-25-04-055.parquet.out','predict-2022-01-13-10-44-07-342.parquet.out']
}

### Fetch Data

##### Fetch realized sales

In [3]:
realized_sales = read_multipart_parquet_s3('fcst-workspace/forecast-cn/fcst-refined-demand-forecast-dev',
                                       'global/model_week_sales', prefix_filename='part-')
realized_sales = realized_sales.groupby(by=['model_id','week_id','date'],as_index=False).agg({'sales_quantity':'sum'})

In [4]:
realized_sales.info(2)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3789624 entries, 0 to 3789623
Data columns (total 4 columns):
 #   Column          Dtype         
---  ------          -----         
 0   model_id        int64         
 1   week_id         int64         
 2   date            datetime64[ns]
 3   sales_quantity  int64         
dtypes: datetime64[ns](1), int64(3)
memory usage: 144.6 MB


##### Fetch APO forecast

In [5]:
apo_gd = read_multipart_parquet_s3('fcst-workspace/forecast-cn/fcst-refined-demand-forecast-dev',
                                       'global/apo_gd_7000.parquet', prefix_filename='part-')

In [6]:
# check data
apo_cutoff = apo_gd['Weeks of snapshot'].unique()
print(f'The apo gd data covers cut_off from {min(apo_cutoff)} to {max(apo_cutoff)}')

apo_period = [int(i) for i in apo_gd.columns if i.startswith('2')]
print(f'The apo gd data covers forecast from {min(apo_period)} to {max(apo_period)}')

The apo gd data covers cut_off from 202138 to 202147
The apo gd data covers forecast from 202138 to 202152


In [7]:
# transform the schema of apo global demand data
cut_w, tar_w = pd.core.reshape.util.cartesian_product([apo_cutoff, apo_period])
w = pd.DataFrame({'cut_off': cut_w, 'target': tar_w})
w = w[(w.target - w.cut_off >= 0)]

def get_value(data,x,y,col):
    df = data[(data['Weeks of snapshot'] == x)][[str(y),'model_id']]
    df[['cut_off','target']] = x,y
    df.columns = [col,'model_id','cut_off','target']
    return df[['cut_off','target','model_id',col]]

fore = pd.DataFrame()
for x, y in w.values:
    tmp_fore = get_value(apo_gd,x,y,'apo_gd')
    fore = fore.append(tmp_fore)

In [8]:
fore.info(2)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 710750 entries, 0 to 67715
Data columns (total 4 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   cut_off   710750 non-null  int64  
 1   target    710750 non-null  int64  
 2   model_id  710750 non-null  int64  
 3   apo_gd    686269 non-null  float64
dtypes: float64(1), int64(3)
memory usage: 27.1 MB


##### Fetch AI forecast result

In [9]:
def keys(bucket_name, prefix='/', delimiter='/'):
    prefix = prefix[1:] if prefix.startswith(delimiter) else prefix
    bucket = boto3.resource('s3').Bucket(bucket_name)
    return (_.key for _ in bucket.objects.filter(Prefix=prefix))

def get_result_path(run_name,bucket_name,cut_off):
    file = []
    cutoff_l = []
    for cutoff in cut_off:
        for i in keys(bucket_name,
                      prefix=f'forecast-cn/fcst-refined-demand-forecast-dev/specific/{run_name}/{run_name}-deepar-hw-{cutoff}/output/',
                      delimiter='/'):
            file.append(i)
            cutoff_l.append(cutoff)
    file_df = pd.DataFrame({'cut_off':cutoff_l,'path':file})
    file_df = file_df[[i.endswith('parquet.out') for i in file_df.path]]
    return file_df

def get_week_id(str_,step):
    str_ = str(str_)
    year = int(str_[:4])
    week = int(str_[4:])
    step = step - 1
    if week + step <= 52:
        if week+step < 10:
            return int(str(year) + '0' +str(week + step))
        else:
            return int(str(year) + str(week + step))
    else:
        round_ = math.floor((week + step) / 52)
        year = year + round_
        week = week + step - 52 * round_
        if week < 10:
            return int(str(year) + '0' + str(week))
        else:
            return int(str(year) + str(week))

In [10]:
# To get all the path of model result
run_name = 'piloted'
ai_cut_off = [i for i in range(202101,202153)] + [i for i in range(202201,202204)]
bucket_name = 'fcst-workspace'
file = get_result_path(run_name,bucket_name,ai_cut_off)

In [ ]:
# if you would like to only filter out results of certain version
version_name = '20220113_145'
file['path_'] = file.path.apply(lambda x:x.split('/')[-1])
file = file[file.path_.isin(version_[version_name])]

In [11]:
ai_forecast = pd.DataFrame()
for w,f in file[['cut_off','path']].values:
    try:
        forecast_df = read_multipart_parquet_s3('fcst-workspace',f, prefix_filename='part-')
        forecast_df['cut_off'] = int(w)
    except:
        forecast_df = pd.DataFrame()
    ai_forecast = ai_forecast.append(forecast_df)
    
ai_forecast = ai_forecast[(ai_forecast.forecast_step <= 10)].groupby(by=['model_id','forecast_step','cut_off'],
                                                  as_index=False).agg({'forecast':'sum'})
ai_forecast['target'] = [get_week_id(x,y) for x,y in zip(ai_forecast['cut_off'], ai_forecast['forecast_step'])]

In [12]:
ai_forecast.info(2)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3951040 entries, 0 to 3951039
Data columns (total 5 columns):
 #   Column         Dtype
---  ------         -----
 0   model_id       int64
 1   forecast_step  int64
 2   cut_off        int64
 3   forecast       int64
 4   target         int64
dtypes: int64(5)
memory usage: 180.9 MB


##### Fetch Price

In [13]:
# fetch price to calculate turnover
price = read_multipart_parquet_s3('fcst-workspace/forecast-cn/fcst-refined-demand-forecast-dev',
                                  'global/model_week_price', prefix_filename='part-')
price = price[price.week_id >= min(ai_cut_off)].groupby(by=['model_id','week_id','date'],as_index=False).agg({'average_price':'mean'})

In [14]:
df_price = price.groupby(by='model_id',as_index=False).agg(initial_price =('average_price','mean'))

In [15]:
df_price.info(2)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24699 entries, 0 to 24698
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   model_id       24699 non-null  int64  
 1   initial_price  24699 non-null  float64
dtypes: float64(1), int64(1)
memory usage: 578.9 KB


In [16]:
to_share = realized_sales[(realized_sales.week_id >= 202101)].groupby(by='model_id',as_index=False).agg(sales_2021=('sales_quantity','sum'))
to_share = to_share.merge(df_price, on='model_id', how='left')
to_share['TO'] = to_share.sales_2021 * to_share.initial_price

In [17]:
to_share['TO'].sum()

11954572821.69491

### Generate features for analysis
For now, we consider:
- selling length
- EC tags
- seasonality

##### Selling length

In [18]:
def selling_week_index(start_week, cut_off):
    try:
        if get_week_id(start_week,51) < cut_off:
            return '>1year'
        elif get_week_id(start_week,19) <= cut_off:
            return '20-52W'
        else:
            return '<20W'
    except:
        return np.nan

In [19]:
start_selling_week = realized_sales.groupby(by='model_id',as_index=False).agg(start_week=('week_id','min'),
                                                                              last_week=('week_id','max')
                                                                             )
start_selling_week['selling_length'] = start_selling_week.start_week.apply(lambda x:selling_week_index(x,202208))

In [20]:
start_selling_week.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48733 entries, 0 to 48732
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   model_id        48733 non-null  int64 
 1   start_week      48733 non-null  int64 
 2   last_week       48733 non-null  int64 
 3   selling_length  48733 non-null  object
dtypes: int64(3), object(1)
memory usage: 1.9+ MB


##### EC tags

In [21]:
def creat_d11_trend2020(df_sales):
    df1 = df_sales[df_sales['week_id'].isin([202045,202046])]
    df2 = df_sales[df_sales['week_id'].isin([202044,202047])]
    df3 = df_sales[df_sales['week_id'].isin([202043,202044,202047,202048])]
    
    sales1 = df1.groupby('model_id',as_index=False).agg(sales_d11 = ('sales_quantity', 'mean'))
    sales2 = df2.groupby('model_id',as_index=False).agg(sales_w2 = ('sales_quantity', 'mean'))
    sales3 = df3.groupby('model_id',as_index=False).agg(sales_w4 = ('sales_quantity', 'mean'))
    
    ec_d11_trend = pd.merge(sales1, sales2, how='left')
    ec_d11_trend = pd.merge(ec_d11_trend, sales3, how='left')
    
    ec_d11_trend['d11_2w'] = ec_d11_trend['sales_d11'] / ec_d11_trend['sales_w2']
    ec_d11_trend['d11_4w'] = ec_d11_trend['sales_d11'] / ec_d11_trend['sales_w4']
    
    ec_d11_trend['ec_tag'] = np.nan
    ec_d11_trend.loc[ec_d11_trend[ec_d11_trend['d11_4w'] >= 1.5].index, 'ec_tag'] = 'High'
    ec_d11_trend.loc[ec_d11_trend[(ec_d11_trend['d11_4w'] < 1.5) & (ec_d11_trend['d11_2w'] > 2)].index, 'ec_tag'] = 'Low'
    ec_d11_trend.loc[ec_d11_trend[ec_d11_trend['ec_tag'].isnull()].index, 'ec_tag'] = 'No'
    
    ec_d11_trend = ec_d11_trend[['model_id', 'ec_tag']]

    return ec_d11_trend

def creat_d11_trend2021(df_sales):
    df1 = df_sales[df_sales['week_id'].isin([202144,202145])]
    df2 = df_sales[df_sales['week_id'].isin([202143,202146])]
    df3 = df_sales[df_sales['week_id'].isin([202142,202143,202146,202147])]
    
    sales1 = df1.groupby('model_id',as_index=False).agg(sales_d11 = ('sales_quantity', 'mean'))
    sales2 = df2.groupby('model_id',as_index=False).agg(sales_w2 = ('sales_quantity', 'mean'))
    sales3 = df3.groupby('model_id',as_index=False).agg(sales_w4 = ('sales_quantity', 'mean'))
    
    base = df_sales[['model_id']].drop_duplicates()
    ec_d11_trend = base.merge(sales3, how='left').merge(sales2,how='left').merge(sales1,how='left')
    
    ec_d11_trend['d11_2w'] = ec_d11_trend['sales_d11'] / ec_d11_trend['sales_w2']
    ec_d11_trend['d11_4w'] = ec_d11_trend['sales_d11'] / ec_d11_trend['sales_w4']
    
    
    ec_d11_trend['ec_tag'] = np.nan
    ec_d11_trend.loc[ec_d11_trend[ec_d11_trend['d11_4w'] >= 1.5].index, 'ec_tag'] = 'High'
    ec_d11_trend.loc[ec_d11_trend[(ec_d11_trend['d11_4w'] < 1.5) & (ec_d11_trend['d11_2w'] > 2)].index, 'ec_tag'] = 'Low'

    ec_d11_trend.loc[ec_d11_trend[ec_d11_trend['ec_tag'].isnull()].index, 'ec_tag'] = 'No'
    
    ec_d11_trend = ec_d11_trend[['model_id', 'ec_tag']]

    return ec_d11_trend

In [22]:
ec_tag = creat_d11_trend2021(realized_sales)

In [23]:
ec_tag.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48733 entries, 0 to 48732
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   model_id  48733 non-null  int64 
 1   ec_tag    48733 non-null  object
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


##### Seasonality label

In [24]:
all_seasonality = realized_sales.groupby(by='model_id',as_index=False).agg({'sales_quantity':['mean','std']})
all_seasonality['annual_seasonality_ratio'] = all_seasonality['sales_quantity']['std'] / all_seasonality['sales_quantity']['mean']
all_seasonality['annual_seasonality'] = ['Normal' if i <= 0.6 else 'Seasonal' for i in all_seasonality['annual_seasonality_ratio']]
all_seasonality.columns = ['model_id','salse_std','sales_mean','annual_seasonality_ratio','annual_seasonality']

aw_seasonality = realized_sales[(realized_sales.week_id >= 202138) & (realized_sales.week_id <= 202152)]\
.groupby(by='model_id',as_index=False).agg({'sales_quantity':['mean','std']})
aw_seasonality['aw_seasonality_ratio'] = aw_seasonality['sales_quantity']['std'] / aw_seasonality['sales_quantity']['mean']
aw_seasonality['aw_seasonality'] = ['Normal' if i <= 0.6 else 'Seasonal' for i in aw_seasonality['aw_seasonality_ratio']]
aw_seasonality.columns = ['model_id','salse_std','sales_mean','aw_seasonality_ratio','aw_seasonality']

In [25]:
all_seasonality.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48733 entries, 0 to 48732
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   model_id                  48733 non-null  int64  
 1   salse_std                 48733 non-null  float64
 2   sales_mean                43736 non-null  float64
 3   annual_seasonality_ratio  43736 non-null  float64
 4   annual_seasonality        48733 non-null  object 
dtypes: float64(3), int64(1), object(1)
memory usage: 2.2+ MB


In [26]:
aw_seasonality.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18967 entries, 0 to 18966
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   model_id              18967 non-null  int64  
 1   salse_std             18967 non-null  float64
 2   sales_mean            16694 non-null  float64
 3   aw_seasonality_ratio  16694 non-null  float64
 4   aw_seasonality        18967 non-null  object 
dtypes: float64(3), int64(1), object(1)
memory usage: 889.1+ KB


### Calculating WPE and WAPE

##### Generate base

In [27]:
realized_sales = realized_sales.rename(columns={'week_id':'target'})

In [28]:
base = ai_forecast[['model_id','cut_off','target','forecast','forecast_step']]\
.merge(realized_sales[['model_id','target','sales_quantity']],on=['model_id','target'],how='left')\
.merge(fore,on=['model_id','cut_off','target'],how='left')\
.merge(df_price,on='model_id',how='left')\
.merge(start_selling_week,on='model_id',how='left')\

In [29]:
base.head(2)

,model_id,cut_off,target,forecast,forecast_step,sales_quantity,apo_gd,initial_price,start_week,last_week,selling_length
0,720719,202101,202101,53,1,25.0,NaN,20.762043,201601,202203,>1year
1,720719,202102,202102,40,1,20.0,NaN,20.762043,201601,202203,>1year


In [30]:
# remove future weeks that could not do the back test
base = base[base.target <= 202203]

# To analyze models still have sales until now
base = base[base.last_week >= 202203]

# remove model_id with no price, which means there was no sales for this model since 2021W1 but the MRP status is active 
base = base[base.initial_price.isna() == False]

# fill N/A sales quantity with 0
base.sales_quantity = base.sales_quantity.fillna(0)

In [31]:
# check base
base.isna().sum()

model_id                0
cut_off                 0
target                  0
forecast                0
forecast_step           0
sales_quantity          0
apo_gd            2617574
initial_price           0
start_week              0
last_week               0
selling_length          0
dtype: int64

##### WPE 4W

In [32]:
def wpe_func(forecast_base,horizon,forecast):
    if forecast == 'apo_gd':
        cutoff = apo_cutoff
    else:
        cutoff = ai_cut_off
    recent_sales = forecast_base[forecast_base.sales_quantity.isna()==False].target.max()
    cutoff = [i for i in cutoff if get_week_id(i,horizon) <= recent_sales]
    wpe = forecast_base[(forecast_base.forecast_step <= horizon) \
                        & (forecast_base.cut_off.isin(cutoff))\
                        & (forecast_base.target <= recent_sales)]\
    .groupby(by=['cut_off','model_id'],as_index=False).agg({'sales_quantity':'sum',
                                                            forecast:'sum',
                                                            'initial_price':'mean'})
    wpe['gap_qty'] = abs(wpe['sales_quantity'] - wpe[forecast])
    wpe['gap_to'] = (wpe['gap_qty'] * wpe['initial_price']).astype(int)
    wpe['wpe_{}W_qty'.format(horizon)] = wpe['gap_qty']/wpe['sales_quantity'] 
    wpe['total_to'] = (wpe['sales_quantity'] * wpe['initial_price']).astype(int)
    wpe = wpe.fillna(0)    
    wpe_all = wpe.groupby(by='cut_off',as_index=False).agg({'gap_qty':'sum','sales_quantity':'sum',
                                                            'gap_to':'sum','total_to':'sum'})
    wpe_all['wpe_{}W_qty'.format(horizon)] = wpe_all['gap_qty']/wpe_all['sales_quantity']
    wpe['wpe_{}W_to'.format(horizon)] = wpe['gap_to']/wpe['total_to']
  
    return wpe, wpe_all


def wpe_all(data,horizon):
    data_ = data.groupby(by=['model_id'],as_index=False).agg({'gap_qty':'sum','sales_quantity':'sum',
                                                          'gap_to':'sum','total_to':'sum'})

    data_['wpe_{}W_qty'.format(horizon)] = data_['gap_qty']/data_['sales_quantity']
    data_['wpe_{}W_to'.format(horizon)] = data_['gap_to']/data_['total_to']
    return data_

In [33]:
# Generate wpe 4W of APO global demand
df_wpe_gd = wpe_func(base,4,'apo_gd')[0]
df_wpe_all_gd = wpe_all(df_wpe_gd,4)
df_wpe_all_gd.columns = ['model_id','gap_qty','sales_quantity','gap_to','total_to','wpe_4W_qty_gd','wpe_4W_to_gd']

In [34]:
# Generate wpe 4W of AI
df_wpe_ai = wpe_func(base,4,'forecast')[0]
df_wpe_all_ai = wpe_all(df_wpe_ai,4)
df_wpe_all_ai.columns = ['model_id','gap_qty','sales_quantity','gap_to','total_to','wpe_4W_qty_ai','wpe_4W_to_ai']

In [35]:
#wpe = pd.pivot_table(data=df_wpe_ai, columns='cut_off', index='model_id', values='wpe_4W_qty')
#wpe = wpe.reset_index()

In [36]:
#df_wpe_ai.to_csv('wpe_all_cutoffs.csv',index=False)

In [ ]:
# save result
#wpe.to_csv('wpe_all_cutoffs.csv',index=False)

##### WAPE 10W

In [37]:
def wape_func(forecast_base,horizon,forecast):
    if forecast == 'apo_gd':
        cutoff = apo_cutoff
    else:
        cutoff = ai_cut_off
    recent_sales = forecast_base[forecast_base.sales_quantity.isna()==False].target.max()
    cutoff = [i for i in cutoff if get_week_id(i,horizon) <= recent_sales]
    wape = forecast_base[(forecast_base.forecast_step <= horizon)\
                         & (forecast_base.cut_off.isin(cutoff))]\
    .groupby(by=['cut_off','model_id','forecast_step'],as_index=False).agg({'sales_quantity':'sum',
                                                                            forecast:'sum','initial_price':'mean'})
    wape['gap_qty'] = abs(wape['sales_quantity'] - wape[forecast])
    wape['gap_to'] = (wape['gap_qty'] * wape['initial_price']).astype(int)
    wape['total_to'] = (wape['sales_quantity'] * wape['initial_price']).astype(int)

    wape['wape_{}W_qty'.format(horizon)] = wape['gap_qty']/wape['sales_quantity']
    wape['wape_{}W_to'.format(horizon)] = wape['gap_to']/wape['total_to']
    
    wape = wape.fillna(0)
    
    wape_all = wape.groupby(by=['model_id','cut_off'],as_index=False).agg({'gap_qty':'sum','sales_quantity':'sum',
                                                                           'gap_to':'sum','total_to':'sum'})
    wape_all['wape_{}W_qty'.format(horizon)] = wape_all['gap_qty']/wape_all['sales_quantity']
    wape_all['wape_{}W_to'.format(horizon)] = wape_all['gap_to']/wape_all['total_to']
    
    return wape, wape_all

def wape_all(data,horizon):
    data_ = data.groupby(by=['model_id'],as_index=False).agg({'gap_qty':'sum','sales_quantity':'sum',
                                                              'gap_to':'sum','total_to':'sum'})

    data_['wape_{}W_qty'.format(horizon)] = data_['gap_qty']/data_['sales_quantity']
    data_['wape_{}W_to'.format(horizon)] = data_['gap_to']/data_['total_to']
    return data_

In [38]:
# Generate wape 10W of APO global demand
df_wape_gd = wape_func(base,10,'apo_gd')[0]
df_wape_all_gd = wape_all(df_wape_gd,10)
df_wape_all_gd.columns = ['model_id','gap_qty','sales_quantity','gap_to','total_to','wape_10W_qty_gd','wape_10W_to_gd']

In [39]:
# Generate wape 10W of AI
df_wape_ai = wape_func(base,10,'forecast')[0]
df_wape_all_ai = wape_all(df_wape_ai,10)
df_wape_all_ai.columns = ['model_id','gap_qty','sales_quantity','gap_to','total_to','wape_10W_qty_ai','wape_10W_to_ai']

In [38]:
#wape = pd.pivot_table(data=wape_func(base,10,'forecast')[1], columns='cut_off', index='model_id', values='wape_10W_qty')
#wape = wape.reset_index()

In [40]:
wape = wape_func(base,10,'forecast')[1]

##### Merge wpe and wape performance

In [41]:
wpe = df_wpe_ai[['model_id','cut_off','gap_qty','gap_to','total_to','sales_quantity']]
wpe.columns = ['model_id','cut_off','wpe_gap_qty','wpe_gap_to','wpe_total_to','wpe_total_sales']

wape = wape[['model_id','cut_off','gap_qty','gap_to','total_to','sales_quantity']]
wape.columns = ['model_id','cut_off','wape_gap_qty','wape_gap_to','wape_total_to','wape_total_sales']

In [42]:
wpe_wape = wpe.merge(wape, on =['model_id','cut_off'], how='left')

In [107]:
wpe_wape.to_csv('wpe_wape.csv',index=False)

##### Analysis

In [69]:
piloted = df_wpe_all_ai[['model_id','wpe_4W_qty_ai','wpe_4W_to_ai']]\
.merge(df_wape_all_ai[['model_id','wape_10W_qty_ai','wape_10W_to_ai']],on='model_id',how='left')\
.merge(start_selling_week,on='model_id',how='left')\
.merge(ec_tag,on='model_id',how='left')\
.merge(aw_seasonality[['model_id','aw_seasonality_ratio','aw_seasonality']],on='model_id',how='left')\
.merge(all_seasonality[['model_id','annual_seasonality_ratio','annual_seasonality']],on='model_id',how='left')\
.merge(to_share[['model_id','TO']],on='model_id',how='left')

In [70]:
# to check if reach target
piloted['wpe_reach_target'] = [True if i <= 0.18 else False for i in piloted.wpe_4W_qty_ai]
piloted['wape_reach_target'] = [True if i <= 0.27 else False for i in piloted.wape_10W_qty_ai]

wpe_reach_target = df_wpe_ai[df_wpe_ai.wpe_4W_qty <= 0.18].groupby(by='model_id',as_index=False)\
.agg(num_wpe_reach_target=('cut_off','count'))

wape_all = wape_func(base,10,'forecast')[1]
wape_reach_target = wape_all[wape_all.wape_10W_qty <= 0.27].groupby(by='model_id',as_index=False)\
.agg(num_wape_reach_target=('cut_off','count'))

wpe_profile = df_wpe_ai.groupby(by='model_id',as_index=False).agg(total_wpe_cnt=('wpe_4W_qty','count'),
                                                                  wpe_std=('wpe_4W_qty','std'),
                                                                  wpe_max=('wpe_4W_qty','max'))

wape_profile = wape_all.groupby(by='model_id',as_index=False).agg(total_wape_cnt=('wape_10W_qty','count'),
                                                                  wape_std=('wape_10W_qty','std'),
                                                                  wape_max=('wape_10W_qty','max'))

In [71]:
# generate tier
def tier_generation(data):
    data['tier'] = np.nan
    data.loc[data[(data.annual_seasonality == 'Normal') & (data.selling_length == '>1year')\
                  & (data.ec_tag == 'No')].index, 'tier'] = 'Tier_1'
    # Tier 2
    data.loc[data[(data.annual_seasonality == 'Normal') & (data.selling_length == '>1year')\
                  & (data.ec_tag.isin(['High','Low']))].index, 'tier'] = 'Tier_21'
    data.loc[data[(data.annual_seasonality == 'Normal') & (data.selling_length == '20-52W')].index, 'tier'] = 'Tier_22'
    # Tier 3
    data.loc[data[(data.annual_seasonality == 'Seasonal') & (data.selling_length == '>1year')].index, 'tier'] = 'Tier_3'
    data.loc[data[(data.annual_seasonality != 'Normal') & (data.aw_seasonality == 'Seasonal')\
                  & (data.selling_length == '>1year')].index, 'tier'] = 'Tier_3'
    # Tier 4
    data.loc[data[(data.annual_seasonality == 'Normal') & (data.selling_length == '<20W')].index, 'tier'] = 'Tier_4'
    data.loc[data[(data.annual_seasonality == 'Seasonal') & (data.selling_length != '>1year')].index, 'tier'] = 'Tier_4'
    return data

piloted = tier_generation(piloted)

In [72]:
# Wheteher 2021 Code A
with open('codeA_2021.yaml') as config_file:
    codea_list = yaml.full_load(config_file)
piloted['AW2021_CodeA'] = [True if i in codea_list['aw2021_codea'] else False for i in piloted.model_id]

In [47]:
# mapping family, sports and RS information
df_tree = read_multipart_parquet_s3('fcst-workspace/forecast-cn/fcst-refined-demand-forecast-dev',
                                    'global/model_week_tree', prefix_filename='part-')
df_tree = df_tree[(df_tree.week_id == 202203)]

rs_info = read_multipart_parquet_s3('fcst-workspace/forecast-cn/fcst-refined-demand-forecast-dev',
                                    'global/family_rs_mapping.parquet', prefix_filename='part-')
rs_info['RS Email Address'] = [i[1] if i[0] is None else i[0] for i in zip(rs_info['RS Email Address'],rs_info['USER Profile'])]
rs_info = rs_info[['Family','RS Email Address']]
rs_info.columns = ['family_id','rs_email']

In [49]:
# fetch MRP status
df_mrp = read_multipart_parquet_s3('fcst-workspace/forecast-cn/fcst-refined-demand-forecast-dev',
                                    'global/model_week_mrp', prefix_filename='part-')
df_mrp = df_mrp[(df_mrp.week_id == 202203)]

In [73]:
piloted = piloted.merge(df_tree[['model_id','model_label','family_id','family_label','univers_label','department_label']], on='model_id', how='left')\
.merge(rs_info, on='family_id', how='left')\
.merge(df_mrp,on='model_id', how='left')\
.merge(wpe_reach_target,on='model_id', how='left')\
.merge(wape_reach_target,on='model_id', how='left')\
.merge(wape_profile,on='model_id', how='left')\
.merge(wpe_profile,on='model_id', how='left')

In [74]:
piloted[['num_wpe_reach_target','num_wape_reach_target',
         'total_wpe_cnt','total_wape_cnt']] = piloted[['num_wpe_reach_target','num_wape_reach_target',
                                                       'total_wpe_cnt','total_wape_cnt']].fillna(0)

In [75]:
piloted.columns

Index(['model_id', 'wpe_4W_qty_ai', 'wpe_4W_to_ai', 'wape_10W_qty_ai',
       'wape_10W_to_ai', 'start_week', 'last_week', 'selling_length', 'ec_tag',
       'aw_seasonality_ratio', 'aw_seasonality', 'annual_seasonality_ratio',
       'annual_seasonality', 'TO', 'wpe_reach_target', 'wape_reach_target',
       'tier', 'AW2021_CodeA', 'model_label', 'family_id', 'family_label',
       'univers_label', 'department_label', 'rs_email', 'week_id',
       'is_mrp_active', 'num_wpe_reach_target', 'num_wape_reach_target',
       'total_wape_cnt', 'wape_std', 'wape_max', 'total_wpe_cnt', 'wpe_std',
       'wpe_max'],
      dtype='object')

In [76]:
piloted.model_label = [str(i[0])+'_'+str(i[1])for i in zip(piloted.model_id,piloted.model_label)]
piloted.family_label = [str(i[0])+'_'+str(i[1])for i in zip(piloted.family_id,piloted.family_label)]

In [77]:
# export to CSV
piloted[['model_id','model_label','TO','is_mrp_active','AW2021_CodeA','rs_email','family_label','univers_label','department_label','tier',
         'wpe_reach_target','wpe_4W_qty_ai','wpe_4W_to_ai','num_wpe_reach_target','total_wpe_cnt','wpe_std','wpe_max',
         'wape_reach_target','wape_10W_qty_ai','wape_10W_to_ai','num_wape_reach_target','total_wape_cnt','wape_std','wape_max',
         'start_week', 'selling_length', 'ec_tag',
         'aw_seasonality_ratio','aw_seasonality', 'annual_seasonality_ratio','annual_seasonality']]\
.to_csv('piloted_analysis.csv',index=False)

In [49]:
data = df_wpe_all_ai.merge(piloted[['model_id','annual_seasonality','selling_length','tier']],on='model_id',how='left')
for dimension in ['annual_seasonality','selling_length','tier']:
    data_ = data.groupby(by=[dimension],as_index=False).agg({'gap_qty':'sum','sales_quantity':'sum'})
    data_['wpe_{}W_all'.format(4)] = data_['gap_qty']/data_['sales_quantity']
    print(data_)

  annual_seasonality     gap_qty  sales_quantity  wpe_4W_all
0             Normal  41022363.0     236308818.0    0.173596
1           Seasonal  66470511.0     245211616.0    0.271074
  selling_length     gap_qty  sales_quantity  wpe_4W_all
0         20-52W   9816008.0      30912203.0    0.317545
1           <20W     59771.0        122404.0    0.488309
2         >1year  97617095.0     450485827.0    0.216693
     tier     gap_qty  sales_quantity  wpe_4W_all
0  Tier_1  29720810.0     179897893.0    0.165209
1  Tier_2  11258530.0      56319973.0    0.199903
2  Tier_3  60891493.0     230623204.0    0.264030
3  Tier_4   5622041.0      14679364.0    0.382989
